## 코사인유사도

In [0]:
!pip install konlpy
!pip install JPype1-py3

     |████████████████████████████████| 19.4MB 230kB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
     |████████████████████████████████| 2.3MB 59.9MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
     |████████████████████████████████| 92kB 2.5MB/s 
  Created wheel for JPype1-py3: filename=JPype1_py3-0.5.5.4-cp36-cp36m-linux_x86_64.whl size=2678108 sha256=cb6c0bca31d22ae86f1d2bd2408da82a9d7c6cc4c92ce28aecf061e4e535173f
  Stored in directory: /root/.cache/pip/wheels/52/37/1f/1015d908d12a0e9b239543d031fda0cded9823aa1306939541
Successfully built JPype1-py3


In [0]:
! bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Installing automake (A dependency for mecab-ko)
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [81.6 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Packages [30.4 kB]
Get:12 http://

In [0]:
# 코드 3-35

import torch
import torch.nn as nn

class CustomRNN(nn.Module):
    def __init__(self, input_size, hidden_size, batch_first=True):
        super(CustomRNN, self).__init__()
        # 선형결합에 사용할 학습가능한 매개변수를 생성한다.
        self.weight_xh, self.weight_hh, self.bias = \
            self.init_weight(input_size, hidden_size)
        # 필요한 정보를 저장한다
        self.hidden_size = hidden_size
        self.batch_first = batch_first
        
    def forward(self, inputs):
        """
        rnn_cell 을 구동하기 위해 inputs 의 크기 (T, B, E) 형태가 되어야 한다.
         - T: 시퀀스 총 길이
         - B: 미니배치크기
         - E: 입력층 크기 
        """
        if self.batch_first:
            # 첫번째 차원이 미니배치 크기인 경우 전치연산으로 바꿔준다.
            inputs = inputs.transpose(0, 1)
        seqlen, batch_size, _ = inputs.size()
        # 0 time-step 에서 은닉층 값을 0으로 초기화 시킨다
        hidden = self.init_hidden(batch_size, self.hidden_size)
        # output에 은닉층의 출력값을 저장한다.
        output = []
        # 시퀀스의 총 길이만큼 순방향전파를 진행한다.
        for i in range(seqlen):
            hidden = self.rnn_cell(inputs[i], hidden)
            output.append(hidden)
        output = torch.stack(output)
        if self.batch_first:
            output = output.transpose(0, 1)
        # 모든 타임스텝의 은닉층 출력값과 마지막 타임 스텝의 은닉층 출력값을 각각 반환한다.
        return output, hidden
    
    def rnn_cell(self, x, h):
        """RNN Cell"""
        h = x.mm(self.weight_xh.t()) + h.mm(self.weight_hh.t()) + self.bias
        return torch.tanh(h)
    
    def init_hidden(self, batch_size, hidden_size):
        """0 타임스텝에서 은닉층의 초기화"""
        return torch.zeros(batch_size, hidden_size)
    
    def init_weight(self, input_size, hidden_size):
        """rnn_cell 의 선형결합을 위한 초기값"""
        weight_xh = torch.randn(hidden_size, input_size).requires_grad_()
        weight_hh = torch.randn(hidden_size, hidden_size).requires_grad_()
        bias = torch.zeros(1, hidden_size).requires_grad_()
        return weight_xh, weight_hh, bias

In [0]:
rnn_layer = nn.RNN(input_size=5, hidden_size=50, batch_first=True, bidirectional=True, num_layers=3)

In [0]:
# 코드 3-32

from konlpy.tag import Mecab
from gensim.models.word2vec import Word2Vec
# paramiko 설치하라는 오류시: 터미널에서 pip install paramiko 실행해준다.

# 품사정보를 "(단어)/(품사정보)" 처럼 함께 저장하기위해 tokenizer 함수를 정의 한다.
mecab = Mecab()
tokenizer = lambda x: ["/".join((tkn.lower(), pos.lower())) for (tkn, pos) in mecab.pos(x)]

with open("ratings_train.txt") as file:
    # 행단위로 데이터를 분리한다. 첫 행은 header라 제외한다.
    raw_data = file.read().splitlines()[1:]
    # 텍스트 데이터만 사용한다
    data = [line.split("\t")[1] for line in raw_data]
    # 토큰화를 진행한다.
    data = [tokenizer(sent) for sent in data]

model = Word2Vec(sentences=data, size=100, window=5, min_count=3, sg=1)
# 훈련 완료후 불필요한 메모리 제거
model.init_sims(replace=True)
# 단어 임베딩 행렬의 크기
print(model.wv.vectors.shape)
# 모델 저장: 파일의 첫번째 줄에는 임베딩 행렬의 크기가 적혀있다. 
model.wv.save_word2vec_format("./word2vec.pt")

print(rnn_layer)

(14409, 100)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


RNN(5, 50, num_layers=3, batch_first=True, bidirectional=True)


In [0]:
# 코드 3-33

# 1. 단어"여배우" 와 "배우"의 유사도
sim1 = model.wv.similarity(*tokenizer("여배우 배우"))
print("similarity(여배우, 배우) = {:.2f}".format(sim1))

similarity(boy, girl) = 0.95


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
# 코드 3-33

# 2. "스토리"와 가장 유사한 단어 Top 5
sim2 = model.wv.most_similar(tokenizer("스토리"), topn=5)
for t, s in sim2:
    print("{} = {:.2f}".format(t, s))

girl/sl = 0.95
son/sl = 0.90
preseason/sl = 0.89
daughter/sl = 0.89
sister/sl = 0.88


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
# 코드 3-33

# 3 벡터 연산 "남자배우" - "남자" = "연기자"
sim3 = model.wv.most_similar(positive=tokenizer("남자배우"), 
                             negative=tokenizer("남자"), 
                             topn=1)
print(sim3)

[('연기력/nng', 0.8554387092590332)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# Mini-Project: 영화 댓글 감성 분류

> 3.2.6 장에 해당하는 코드

In [0]:
! bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [0]:
import torch
import torch.nn.functional as F 

def train(model, train_loader, loss_func, optimizer, step, print_step=200):
    """train function"""
    model.train()
    for i, batch in enumerate(train_loader):
        inputs, targets = batch.text, batch.label.float()
        # 경사 초기화
        optimizer.zero_grad()
        # 순방향 전파
        outputs = model(inputs)
        # 손실값 계산
        loss = loss_func(outputs, targets)
        # 역방향 전파
        loss.backward()
        # 매개변수 업데이트
        optimizer.step()
    
        if i % print_step == 0:
            print('Train Step: {} ({:05.2f}%)  \tLoss: {:.4f}'.format(
                    step, 100.*(i*train_loader.batch_size)/len(train_loader.dataset), 
                    loss.item()))

def test(model, test_loader, loss_func):
    """test function"""
    # 모델에게 평가단계이라고 선언함
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = batch.text, batch.label.float()
            # 순방향전파
            outputs = model(inputs)
            # 손실값 계산(합)
            test_loss += loss_func(outputs, targets, reduction="sum").item()
            # 예측값
            preds = torch.sigmoid(outputs).ge(0.5).float()
            # 정확하게 예측한 개수를 기록한다
            correct += preds.eq(targets).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_acc = correct / len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:05.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset), 100. * test_acc))
    return test_loss, test_acc

def main(model, train_loader, test_loader, loss_func, optimizer, n_step, 
         save_path=None, print_step=30):
    """메인 학습 함수"""
    test_accs = []
    best_acc = 0.0

    for step in range(1, n_step+1):
        # 훈련 단계
        train(model, train_loader, loss_func, optimizer, 
              step=step, print_step=print_step)
        # 평가 단계
        test_loss, test_acc = test(model, test_loader, 
                                   loss_func=F.binary_cross_entropy_with_logits)
        # 테스트 정확도 기록
        test_accs.append(test_acc)
        # 모델 최적의 매개변수값을 저장할지 결정하고 기록한다.
        if len(test_accs) >= 2:
            if test_acc >= best_acc:
                best_acc = test_acc
                best_state_dict = model.state_dict()
                print("discard previous state, best model state saved!")
        print("")

    # 매개변수 값 저장하기
    if save_path is not None:
        torch.save(best_state_dict, save_path)

## 패키지 로딩, 데이터 전처리

In [0]:
# 코드 3-40

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, TabularDataset, Iterator
from konlpy.tag import Mecab

# 토큰화 함수로 MeCab 사용
tokenizer = Mecab()

# 필드 정의
TEXT = Field(sequential=True,
             use_vocab=True,
             tokenize=tokenizer.morphs,  
             lower=True, 
             batch_first=True)  
LABEL = Field(sequential=False,  
              use_vocab=False,   
              preprocessing = lambda x: int(x),
              batch_first=True, 
              is_target=True)

# 각 댓글에 해당하는 id, 사용하지 않지만 기본필드로 정의 해준다.
ID = Field(sequential=False,  
           use_vocab=False,   
           is_target=False)

# TabularDataset.splits 함수를 사용해 훈련 세트와 테스트 세트를 나눈다.
train_data, test_data = TabularDataset.splits(
    path='', format='tsv', 
    train="ratings_train.txt",
    test="ratings_test.txt",
    fields=[('id', ID), ('text', TEXT), ('label', LABEL)],
    skip_header=True)

# 단어장 생성
TEXT.build_vocab(train_data, min_freq=2)
# 데이터 개수 및 단어장 크기 확인
print("Train Data: {} / Test Data: {}".format(len(train_data), len(test_data)))
print("Vocab Size: {}".format(len(TEXT.vocab)))

# 환경 변수 설정
BATCH = 256  # 미니배치크기
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # 디바이스
STEP = 10  # 총 반복스텝

# 데이터 로더 정의
train_loader = Iterator(dataset=train_data, batch_size=BATCH, device=DEVICE)
test_loader = Iterator(dataset=test_data, batch_size=BATCH, device=DEVICE)

# 영화 댓글 감성 분류 모델

In [0]:
# 코드 3-41

class SentimentCls(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size,
                 num_layers=3, batch_first=True, bidirec=True):
        super(SentimentCls, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = num_layers
        self.n_direct = 2 if bidirec else 1
        self.embedding_layer = nn.Embedding(vocab_size, embed_size)
        self.rnn_layer = nn.LSTM(input_size=embed_size,
                                 hidden_size=hidden_size,
                                 num_layers=num_layers,
                                 batch_first=batch_first,
                                 bidirectional=bidirec)
        self.linear = nn.Linear(self.n_direct*hidden_size, output_size)

    def forward(self, x):
        embeded = self.embedding_layer(x)
        hidden, cell = self.init_hiddens(x.size(0), self.hidden_size, device=x.device)
        output, (hidden, cell) = self.rnn_layer(embeded, (hidden, cell))
        last_hidden = torch.cat([h for h in hidden[-self.n_direct:]], dim=1)
        scores = self.linear(last_hidden)
        return scores.view(-1)
    
    def init_hiddens(self, batch_size, hidden_size, device):
        hidden = torch.zeros(self.n_direct*self.n_layers, batch_size, hidden_size)
        cell = torch.zeros(self.n_direct*self.n_layers, batch_size, hidden_size)
        return hidden.to(device), cell.to(device)

## 모델, 손실함수 및 옵티마이저 선언

In [0]:
# 코드 3-42

# 모델 선언에 필요한 인자 설정
vocab_size = len(TEXT.vocab)  # V: 단어장 크기
embed_size = 128  # E: 임베딩 크기
hidden_size = 256  # D: 은닉층 크기
output_size = 1  # 출력층 크기
num_layers = 3  # RNN 층의 개수
batch_first = True  # RNN 입력의 첫번째 차원이 미니배치 크기인 경우 활성화
bidirec = True  # 양방향 순환 신경망 사용 여부

# 모델 선언
model = SentimentCls(vocab_size, embed_size, hidden_size, output_size,
                     num_layers, batch_first, bidirec).to(DEVICE)
# 매개변수 개수 확인하기
num_params = 0
for params in model.parameters():
    num_params += params.view(-1).size(0)
print("Total number of parameters: {}".format(num_params))

# 손실함수와 옵티마이저 선언
loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())

## 학습 및 테스트

In [0]:
# 코드 3-43

# 모델 훈련
main(model=model,
     train_loader=train_loader,
     test_loader=test_loader,
     loss_func=loss_function, 
     optimizer=optimizer, 
     n_step=STEP,
     save_path="./sent_cls.pt",
     print_step=256)

In [0]:
import torch
import torch.nn as nn

class SentimentCls(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size,
                 num_layers=3, batch_first=True, bidirec=True):
        super(SentimentCls, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = num_layers
        self.n_direct = 2 if bidirec else 1
        self.embedding_layer = nn.Embedding(vocab_size, embed_size)
        self.rnn_layer = nn.LSTM(input_size=embed_size,
                                 hidden_size=hidden_size,
                                 num_layers=num_layers,
                                 batch_first=batch_first,
                                 bidirectional=bidirec)
        self.linear = nn.Linear(self.n_direct*hidden_size, output_size)

    def forward(self, x):
        embeded = self.embedding_layer(x)
        hidden, cell = self.init_hiddens(x.size(0), self.hidden_size, device=x.device)
        output, (hidden, cell) = self.rnn_layer(embeded, (hidden, cell))
        last_hidden = torch.cat([h for h in hidden[-self.n_direct:]], dim=1)
        scores = self.linear(last_hidden)
        return scores.view(-1)
    
    def init_hiddens(self, batch_size, hidden_size, device):
        hidden = torch.zeros(self.n_direct*self.n_layers, batch_size, hidden_size)
        cell = torch.zeros(self.n_direct*self.n_layers, batch_size, hidden_size)
        return hidden.to(device), cell.to(device)

## 실제 댓글로 테스트 해보기

In [0]:
import torch
import torch.nn as nn
from torchtext.data import Field, TabularDataset, Iterator
from konlpy.tag import Mecab

## 전처리
# 토큰화 함수로 MeCab 사용
tokenizer = Mecab()

# 필드 정의
TEXT = Field(sequential=True,
             use_vocab=True,
             tokenize=tokenizer.morphs,  
             lower=True, 
             batch_first=True)  
LABEL = Field(sequential=False,  
              use_vocab=False,   
              preprocessing = lambda x: int(x),
              batch_first=True, 
              is_target=True)

# 각 댓글에 해당하는 id, 사용하지 않지만 기본필드로 정의 해준다.
ID = Field(sequential=False,  
           use_vocab=False,   
           is_target=False)

# TabularDataset.splits 함수를 사용해 훈련 세트와 테스트 세트를 나눈다.
train_data, test_data = TabularDataset.splits(
    path='', format='tsv', 
    train="ratings_train.txt",
    test="ratings_test.txt",
    fields=[('id', ID), ('text', TEXT), ('label', LABEL)],
    skip_header=True)

# 단어장 생성
TEXT.build_vocab(train_data, min_freq=2)

## 모델 불러오기
# 모델 선언에 필요한 인자 설정
vocab_size = len(TEXT.vocab)  # V: 단어장 크기
embed_size = 128  # E: 임베딩 크기
hidden_size = 256  # D: 은닉층 크기
output_size = 1  # 출력층 크기
num_layers = 3  # RNN 층의 개수
batch_first = True  # RNN 입력의 첫번째 차원이 미니배치 크기인 경우 활성화
bidirec = True  # 양방향 순환 신경망 사용 여부
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # 디바이스

# 모델 선언 및 불러오기
model = SentimentCls(vocab_size, embed_size, hidden_size, output_size,
                     num_layers, batch_first, bidirec).to(DEVICE)
model.load_state_dict(torch.load("./sent_cls.pt"))
print("Load Complete!")

# 실제 테스트
def test_input(model, field, tokenizer, device):
    sentence = input("테스트할 댓글 작성: ")
    x = field.process([tokenizer.morphs(sentence)]).to(device)
    output = model(x)
    pred = torch.sigmoid(output).ge(0.5).item()
    print("---결과---")
    if pred == 1:
        print("긍정")
    else:
        print("부정")
        
test_input(model, field=TEXT, tokenizer=tokenizer, device=DEVICE)